###### Importando as bibliotecas necessárias

In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from parsel import Selector
import pandas as pd
from time import sleep

###### Abrir página e mudar para o iframe

In [30]:
# Função que inicia o driver:
def iniciar_driver():
    chrome_options = Options()
    # idioma português Brasil e abrir maximiado a tela
    arguments = ['--lang=pt-BR', '--start-maximized']
    for argument in arguments:
        chrome_options.add_argument(argument)
    driver = webdriver.Chrome(options=chrome_options)
    return driver

In [31]:
# inicar driver
site = "https://johnssimplesmentegostoso.com.br/"
driver = iniciar_driver()

In [32]:
# entrar no site
driver.get(site)

In [33]:
# Mudar para o iframe
sleep(3)
driver.switch_to.frame("main")

###### Fazer o Scraping

In [34]:
# Dicionário para armazenar os items
items = {
    "Nome" : [],
    "Categoria" : [],
    "Preço" : []
}

In [35]:
# Análise do código fonte da página
resposta = Selector(driver.page_source)

**Destaques**

In [36]:
destaques = resposta.xpath("//div[@class='highlight-items-carousel']")
# Nome da categoria do item
titulo = destaques.xpath("h2/text()").get()
nomes = destaques.xpath("//h4[@class='name']/text()").extract()
precos = destaques.css("div.prices .current::text").extract()
precos = list(map(lambda p: p.replace("R$\xa0", "").replace(",", "."), precos))
# Atualiza o dicionário items que se encontra vazio:
items.update(
    {"Nome" : [*nomes],
    "Categoria" : [titulo]*len(nomes),
    "Preço" : [*precos]}
)

In [37]:
items

{'Nome': ['Pudim', 'IRRESISTIVEL'],
 'Categoria': ['Destaques', 'Destaques'],
 'Preço': ['12.00', '16.20']}

**Outros Itens**

In [38]:
categorias = resposta.css("div.category-container")
for categoria in categorias:
    titulo = categoria.css('h2.title::text').extract_first()
    nomes = categoria.css("div.text-info h4::text").extract()
    precos = categoria.css("div.price p::text").extract()
    precos = list(map(lambda p: p.replace("R$\xa0", "").replace(",", "."), precos))
    items["Nome"] += nomes
    items["Categoria"] += [titulo]*len(nomes)
    items["Preço"] += precos

**Criando o Data Frame**

In [39]:
items = pd.DataFrame(data=items)

In [40]:
items

,Nome,Categoria,Preço
0,Pudim,Destaques,12.00
1,IRRESISTIVEL,Destaques,16.20
2,GORGON ONION CRISPY,>>> Burger do mês <<<,42.00
3,IRRESISTIVEL,BARATI$$IMO SMASH DAY,18.00
4,OPS ! BACON,BARATI$$IMO SMASH DAY,16.20
5,MELT BACON,BARATI$$IMO SMASH DAY,20.00
6,SMASH BACON,BARATI$$IMO SMASH DAY,22.00
7,BATATA D´CUPIM,PRA COMEÇAR,20.00
8,HOT WINGS,PRA COMEÇAR,34.00
9,BATATA CHEDDAR E CRISPY DE BACON,PRA COMEÇAR,28.00


In [41]:
# Converter preço para o tipo numérico
items["Preço"] = pd.to_numeric(items["Preço"])

In [42]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Nome       45 non-null     object 
 1   Categoria  45 non-null     object 
 2   Preço      45 non-null     float64
dtypes: float64(1), object(2)
memory usage: 1.2+ KB


**Salvando os dados em JSON**

In [54]:
items.to_json("items.json", index=False)

**Abrindo e conferindo o JSON**

In [55]:
items_salvos = pd.read_json("items.json")

In [56]:
items_salvos

,Nome,Categoria,Preço
0,Pudim,Destaques,12.0
1,IRRESISTIVEL,Destaques,16.2
2,GORGON ONION CRISPY,>>> Burger do mês <<<,42.0
3,IRRESISTIVEL,BARATI$$IMO SMASH DAY,18.0
4,OPS ! BACON,BARATI$$IMO SMASH DAY,16.2
5,MELT BACON,BARATI$$IMO SMASH DAY,20.0
6,SMASH BACON,BARATI$$IMO SMASH DAY,22.0
7,BATATA D´CUPIM,PRA COMEÇAR,20.0
8,HOT WINGS,PRA COMEÇAR,34.0
9,BATATA CHEDDAR E CRISPY DE BACON,PRA COMEÇAR,28.0
